# Data Reading Process

Importing libraries

In [1]:
import dask
import pandas as pd
import dask.dataframe as dd
import os
from dask.distributed import Client, LocalCluster
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

Declaring local cluster instance to manipulate and process parallel data

In [2]:
cluster = LocalCluster(n_workers = 8, threads_per_worker = 2, memory_limit = '8GB')
client = Client(cluster) 

Show Dashboard link

In [3]:
client.dashboard_link

'http://127.0.0.1:8787/status'

creating a function to restart client

Importing databases to use to create de machine learning model

In [4]:
# reading all tables from database
table_names = os.listdir('databases')

# importing all databases
tables = {tbl_name.split('.')[0]: client.persist(dd.read_csv(f"databases/{tbl_name}", blocksize ='100MB')) for tbl_name in table_names}

# Data Wrangling

Joining tables to create a single one to use as the main dataframe

In [5]:
tables['liabilities'].head()

,month,ID,product_1,product_2,period
0,11,c7a6cecac361a970ee6c7cb9f2679c38c668940c6ebdc5...,4.1900,0.0,period_5
1,12,68f89dc7efe7cd1ce3cf6d52d8867bd2b84f236736bf91...,4.4621,0.0,period_5
2,2,e3f6afb18792d0b97cb9a63ad9567bbd106146c78d212a...,4.2878,0.0,period_5
3,1,8993170400939d522bdc72707527f5ab90202119fe3cab...,5.0000,5.0,period_5
4,7,399d686897232a55c713d61a6bdc7e232ab85b864d6288...,3.8922,0.0,period_5


In [6]:
tables['liabilities']=tables['liabilities'].rename(columns={'product_1':'product_1_lia',
                                                            'product_2':'product_2_lia'})

In [7]:
# merging all tables
data = tables['balances'].merge(tables['liabilities'], on = ['ID', 'period', 'month'], how = 'left')\
                        .merge(tables['movements'], on = ['ID', 'period', 'month'], how = 'left')\
                        .merge(tables['customers'], on = ['ID'], how = 'left')\
                        .merge(tables['digital'], on = ['ID', 'period', 'month'], how = 'left').repartition(25)

# removing from memory singular tables by deleting
del tables

Showing five records

In [8]:
data.head()

,month,ID,type,product,entity,balance_amount,days_default,period,product_1_lia,product_2_lia,...,dig_2,dig_3,dig_4,dig_5,dig_6,dig_7,dig_8,dig_9,dig_10,dig_11
0,12,01a1b0d7506e7b330b88059d2b9490f5e990d5535f7c51...,type_4,product_4,entity_2,4.3441,0.0000,period_2,3.3049,0.0,...,3.4067,2.4075,2.5198,1.2234,3.0985,3.2626,2.8074,3.7402,2.2106,3.4946
1,9,e8fe36a43e1333e858bd9dab918163ed930e2af49daf68...,type_4,product_2,entity_4,4.0639,0.0000,period_2,3.7750,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,28b29359b46f62dea9f6aba92995d4335eba81d5401ff0...,type_4,product_2,entity_2,3.2628,0.0000,period_2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,39a0d9ad909c97192d1fc8af591b254761ab9162e3eaa8...,type_4,product_2,entity_3,4.4232,0.0000,period_2,3.5224,0.0,...,3.7396,2.5910,2.2634,0.6559,3.9182,4.7796,3.3219,4.4265,2.7894,4.3147
4,5,66e73fb19d87327c19e815b1e84653ac1a7ae5d8cd7975...,type_3,product_1,entity_1,4.0154,4.9231,period_2,3.9162,0.0,...,2.8562,2.2041,1.5450,0.1312,2.4796,5.0000,4.8580,5.0000,2.2106,5.0000


## Null Values Imputation

Checking data types and null values

In [9]:
dd.concat([data.dtypes, data.isnull().sum()/ len(data)], axis = 1).rename(columns = {0: 'data_type', 1: 'null_ratio'}).compute()

,data_type,null_ratio
ID,string,0.000000
age,float64,0.000000
balance_amount,float64,0.000000
bureau_risk,string,0.000000
days_default,float64,0.000000
dig_1,float64,0.247692
dig_10,float64,0.247692
dig_11,float64,0.247692
dig_2,float64,0.247692
dig_3,float64,0.247692


Before deciding what kind of impution it is going to be performed, we observe its quartils to decide if imputing with mean or median

In [10]:
# selecting group of columns based on their name
dig_columns = [column for column in data.columns if 'dig' in column]
prod_columns = ['product_1_lia','product_2_lia']
type_columns = ['type_1', 'type_2', 'type_3', 'type_4']

# describing selected columns
data[dig_columns + prod_columns + type_columns].describe().compute()

,dig_1,dig_2,dig_3,dig_4,dig_5,dig_6,dig_7,dig_8,dig_9,dig_10,dig_11,product_1_lia,product_2_lia,type_1,type_2,type_3,type_4
count,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,8.725112e+06,1.041170e+07,1.041170e+07,5.199137e+06,5.199137e+06,5.199137e+06,5.199137e+06
mean,1.729927e+00,3.250468e+00,3.033453e+00,2.620164e+00,1.598258e+00,3.604106e+00,2.253315e+00,2.012096e+00,3.075644e+00,1.535400e+00,2.711378e+00,3.038695e+00,5.321534e-01,2.075979e+00,2.599862e+00,1.599407e+00,1.741834e+00
std,1.181385e+00,8.854857e-01,8.896423e-01,1.073410e+00,1.103790e+00,1.116196e+00,1.490340e+00,1.398692e+00,1.057191e+00,1.587534e+00,1.256354e+00,1.209978e+00,1.316910e+00,1.884979e+00,1.751632e+00,1.857044e+00,1.931270e+00
min,0.000000e+00,0.000000e+00,-1.000000e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.050000e-01,2.813500e+00,2.526800e+00,1.949600e+00,8.656000e-01,3.226900e+00,1.029200e+00,1.063700e+00,2.431100e+00,0.000000e+00,1.941900e+00,2.463500e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.555900e+00,3.396900e+00,3.107200e+00,2.519800e+00,1.391400e+00,3.870150e+00,2.389800e+00,2.058500e+00,3.187100e+00,1.313200e+00,2.833700e+00,3.217600e+00,0.000000e+00,2.576200e+00,3.189600e+00,0.000000e+00,0.000000e+00
75%,2.386400e+00,3.893900e+00,3.666400e+00,3.238200e+00,2.151500e+00,4.335100e+00,3.429000e+00,3.061500e+00,3.878800e+00,2.789400e+00,3.660300e+00,3.904500e+00,0.000000e+00,3.806425e+00,3.993650e+00,3.451400e+00,3.707700e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


As it can be seen, data has no skew apparently beacuse of the proximity between max value and quartil 3, and mean an median are quite the same, so median is chosen to be de imputer

In [11]:
# Transforming dask to pandas df
df=data.compute()

In [12]:
# joining all selected columns
missing_numeric_columns = dig_columns + prod_columns + type_columns
# declaring column trasnformer
column_transformer = make_column_transformer((SimpleImputer(strategy = 'mean'), missing_numeric_columns), 
                                            remainder = 'passthrough',
                                            n_jobs=-1).set_output(transform = 'pandas')

# imputing values
data_tf = column_transformer.fit_transform(df)

Verifying all nule values are filled

In [13]:
data_tf.isnull().sum()

simpleimputer__dig_1                0
simpleimputer__dig_2                0
simpleimputer__dig_3                0
simpleimputer__dig_4                0
simpleimputer__dig_5                0
simpleimputer__dig_6                0
simpleimputer__dig_7                0
simpleimputer__dig_8                0
simpleimputer__dig_9                0
simpleimputer__dig_10               0
simpleimputer__dig_11               0
simpleimputer__product_1_lia        0
simpleimputer__product_2_lia        0
simpleimputer__type_1               0
simpleimputer__type_2               0
simpleimputer__type_3               0
simpleimputer__type_4               0
remainder__month                    0
remainder__ID                       0
remainder__type                     0
remainder__product                  0
remainder__entity                   0
remainder__balance_amount           0
remainder__days_default             0
remainder__period                   0
remainder__age                      0
remainder__i

Deleting data from memory, no need in following cells

In [14]:
del data

Adding target

In [15]:
#test=pd.read_csv('datasets/universe_test.csv')
train=pd.read_csv('datasets/universe_train.csv')


In [ ]:
# Get a list of column names
columns = data_tf.columns

# Remove any text before '__' in the column names
new_names = [col.split('__', 1)[-1] for col in columns]

# Assign the new column names to the DataFrame
data_tf.columns = new_names

#joining dataframe with target
data_tf=data_tf.merge(train,on=['ID','period'],how='left')

In [ ]:
#exporting data
data_tf.to_csv('datasets/clean_table.csv',index=False)

train=data_tf.loc[~data_tf['attrition'].isna()]
sub=data_tf.loc[data_tf['attrition'].isna()]

train.to_csv('datasets/train_table.csv',index=False)
sub.to_csv('datasets/submission_table.csv',index=False)